# Objective: Implement multi head attention from scratch
- read attention is all you need paper & medium references
- code multi head attention
- code self attention / scaled dot product

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import math
import copy
import numpy as np
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())

CUDA available: True
Device count: 1


## Components of Transfomer: Multi head attention , FFN , Positional Encoding

In [36]:
class MultiHeadAttention(nn.Module):
    def __init__(self,embed_dim,num_heads):
        super(MultiHeadAttention,self).__init__()
        assert embed_dim % num_heads == 0, "d_model must be divisible by num_heads"

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.batch_size = 64

        self.query = nn.Linear(embed_dim,embed_dim)
        self.key = nn.Linear(embed_dim,embed_dim)
        self.value = nn.Linear(embed_dim,embed_dim)
        self.out = nn.Linear(embed_dim,embed_dim)

    def forward(self,qx,kx,vx,mask=None):
        q = self.query(qx).view(self.batch_size,-1,self.num_heads,self.head_dim).transpose(1,2)
        k = self.key(kx).view(self.batch_size,-1,self.num_heads,self.head_dim).transpose(1,2)
        v = self.value(vx).view(self.batch_size,-1,self.num_heads,self.head_dim).transpose(1,2)

        attention_scores = torch.matmul(q,k.transpose(-2,-1)) / np.sqrt(self.head_dim)
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0,-1e9)

        attention_weights = F.softmax(attention_scores,dim=-1)

        

        context = torch.matmul(attention_weights,v).transpose(1,2).contiguous().view(self.batch_size,-1,self.embed_dim)

        return self.out(context)


In [37]:
class FeedForwardNetwork(nn.Module):
    def __init__(self,d_model,d_ff):
        super(FeedForwardNetwork,self).__init__()
        self.fc1 = nn.Linear(d_model,d_ff)
        self.fc2 = nn.Linear(d_ff,d_model)
        self.activation = nn.ReLU()


    def forward(self,x):
        return self.fc2(self.activation(self.fc1(x)))

In [ ]:
# Two types of PE: sinusoidal PE or learnable PE
class PositionalEncoding(nn.Module):
    def __init__(self,d_model,max_seq_length):
        super(PositionalEncoding,self).__init__()

        position_encodings = torch.zeros(max_seq_length,d_model)
        position = torch.arange(0,max_seq_length,dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0,d_model,2).float() * -(math.log(10000.0)/d_model))

        position_encodings[:, 0::2] = torch.sin(position * div_term)
        position_encodings[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe',position_encodings.unsqueeze(0))

    def forward(self,x):
        return x + self.pe[:,:x.size(1)]
    

class LearnablePositionalEncoding(nn.Module):
    def __init__(self,seq_len,embed_dim):
        super(LearnablePositionalEncoding,self).__init__()
        self.positional_encoding = nn.Parameter(torch.zeros(seq_len,embed_dim))

    def forward(self,x):
        return x + self.positional_encoding


## Encoder component

In [39]:
class EncoderLayer(nn.Module):
    def __init__(self,d_model,num_heads,d_ff,dropout):
        super(EncoderLayer,self).__init__()
        self.self_attention = MultiHeadAttention(d_model,num_heads)
        self.feed_forward = FeedForwardNetwork(d_model,d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)


    def forward(self,x,mask):
        attention_output = self.self_attention(x,x,x,mask)
        x = self.norm1(x + self.dropout(attention_output))
        
        ff_output = self.feed_forward(x)
        x = self.norm2(x+self.dropout(ff_output))
        return x

## Decoder Component

In [40]:
class DecoderLayer(nn.Module):
    def __init__(self,d_model,num_heads,d_ff,dropout):
        super(DecoderLayer,self).__init__()
        self.self_attention = MultiHeadAttention(d_model,num_heads)
        self.cross_attention = MultiHeadAttention(d_model,num_heads)
        self.feed_forward = FeedForwardNetwork(d_model,d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x,enc_output,source_mask,target_mask):
        attention_output = self.self_attention(x,x,x,target_mask)
        x = self.norm1(x+self.dropout(attention_output))

        attention_output = self.cross_attention(x,enc_output,enc_output,source_mask)
        x = self.norm2(x+self.dropout(attention_output))

        ff_output = self.feed_forward(x)
        x = self.norm3(x+self.dropout(ff_output))

        return x


## Full transformer

In [41]:
class Transformer(nn.Module):
    def __init__(self,source_vocab_size,target_vocab_size,d_model,num_heads,num_layers,d_ff,max_seq_length,dropout):
        super(Transformer,self).__init__()

        self.encoder_embedding = nn.Embedding(source_vocab_size,d_model)
        self.decoder_embedding = nn.Embedding(target_vocab_size,d_model)
        self.positional_encoding = PositionalEncoding(d_model,max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model,num_heads,d_ff,dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model,num_heads,d_ff,dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model,target_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self,source,target):
        source_mask = (source != 0).unsqueeze(1).unsqueeze(2)
        target_mask = (target != 0).unsqueeze(1).unsqueeze(3)

        seq_length = target.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1,seq_length,seq_length),diagonal=1)).bool()

        target_mask = target_mask & nopeak_mask
        return source_mask , target_mask
    
    def forward(self,source,target):
        source_mask , target_mask = self.generate_mask(source,target)

        source_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(source)))
        target_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(target)))

        encoder_output = source_embedded
        for encoder_layer in self.encoder_layers:
            encoder_output = encoder_layer(encoder_output,source_mask)

        decoder_output = target_embedded
        for decoder_layer in self.decoder_layers:
            decoder_output = decoder_layer(decoder_output,encoder_output,source_mask,target_mask)


        #this output layer changes acorrding to task like NER/sentiment analysis
        output = self.fc(decoder_output)
        return output

## Test Transformer

In [42]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1
batch_size = 64

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (batch_size, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (batch_size, max_seq_length))  # (batch_size, seq_length)

In [45]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

default_loss = -np.log(1/tgt_vocab_size)
print(f"Default loss for vocab size {tgt_vocab_size}: {default_loss:.4f}")

for epoch in range(10):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Default loss for vocab size 5000: 8.5172
Epoch: 1, Loss: 7.412601470947266
Epoch: 2, Loss: 7.616613388061523
Epoch: 3, Loss: 7.388408184051514
Epoch: 4, Loss: 7.291308879852295
Epoch: 5, Loss: 7.210090637207031
Epoch: 6, Loss: 7.132160186767578
Epoch: 7, Loss: 7.055497646331787
Epoch: 8, Loss: 7.001845836639404
Epoch: 9, Loss: 6.9333176612854
Epoch: 10, Loss: 6.859369277954102
